# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# PlannerStaticPipeline — E2E Notebook (STATIC pipeline)

## Scope
We validate **PlannerStaticPipeline** end-to-end after integrating it with **PlanLoopController**.

What we care about in this notebook:
- Pipeline runs deterministic **setup steps** and then delegates planning/execution to **PlanLoopController**.
- Pipeline returns a valid **RuntimeAnswer** that matches the runtime contract.
- End-to-end behavior matches the already validated PlanLoopController semantics (COMPLETED / CLARIFY / FAILED).

## Constraints (this session)
- We add **one notebook cell per message** (including its English markdown).
- Each test is **autonomous** (no shared state between cells).
- We use **only Intergrax framework classes** (no fake adapters, no fake classes, no new custom structures).
- Tests are **production-relevant** (no “empty” or meaningless assertions).
- Determinism is achieved via **EnginePlanner forced_plan** where needed.


## Test 1 — STATIC pipeline Happy Path (COMPLETED)

This test validates that:

- `PlannerStaticPipeline` can be executed end-to-end.
- It delegates planning/execution to `PlanLoopController`.
- A deterministic `forced_plan` produces a `COMPLETED` path.
- The pipeline returns a valid `RuntimeAnswer` matching the runtime contract.

This is a production-relevant scenario:
a normal user request is processed without replans, without clarifications, and without failures.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime_state import RuntimeState

from intergrax.runtime.nexus.pipelines.planner_static_pipeline import PlannerStaticPipeline

from intergrax.runtime.nexus.planning.engine_plan_models import EnginePlan, PlannerPromptConfig

from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


async def test_static_pipeline_clarify_path():
    # ---------------------------------------------------------------------
    # Config (production components only)
    # ---------------------------------------------------------------------
    cfg = RuntimeConfig(
        llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
        enable_rag=False,
        enable_websearch=False,
        tools_mode="off",
    )

    # If your RuntimeConfig requires explicit StepPlanner/Executor configs (common in your code),
    # you should set them here exactly as in your existing PlanLoopController notebook.
    # This cell is strict: we do not create new fake configs.

    # ---------------------------------------------------------------------
    # Deterministic forced plan (dict, not EnginePlan)
    # Must match EnginePlanner parser contract.
    # ---------------------------------------------------------------------
    cfg.planner_prompt_config = PlannerPromptConfig(
    forced_plan=EnginePlan.generic_finalize(
        reasoning_summary="Deterministic happy path plan for STATIC pipeline integration test."
    )
)

    cfg.plan_loop_policy = PlanLoopPolicy()
    cfg.step_planner_cfg = StepPlannerConfig()
    cfg.step_executor_cfg = StepExecutorConfig()

    # ---------------------------------------------------------------------
    # Runtime context + request + state
    # ---------------------------------------------------------------------
    session_manager = SessionManager(storage=InMemorySessionStorage())

    ctx = RuntimeContext.build(
        config=cfg,
        session_manager=session_manager,
    )

    request = RuntimeRequest(
        user_id="test-user",
        session_id="static-pipeline-e2e-001",
        message="Explain what Intergrax is in one short paragraph.",
        attachments=[],
    )

    state = RuntimeState(
        context=ctx,
        request=request,
        run_id="static-pipeline-e2e-run-001",
    )
    state.configure_llm_tracker()

    # Capabilities ON (avoid planner clamp noise; forced_plan already bypasses LLM planning)
    state.cap_websearch_available = True
    state.cap_tools_available = True
    state.cap_rag_available = True
    state.cap_user_ltm_available = True

    # ---------------------------------------------------------------------
    # Run pipeline
    # ---------------------------------------------------------------------
    pipeline = PlannerStaticPipeline()
    answer = await pipeline.run(state)

    # ---------------------------------------------------------------------
    # Assertions (production meaningful)
    # ---------------------------------------------------------------------
    assert answer is not None, "Pipeline returned None."
    assert state.runtime_answer is answer, "state.runtime_answer was not set to the returned answer."
    assert answer.answer and isinstance(answer.answer, str), "Final answer is empty or invalid."
    assert answer.route is not None, "Route is missing."

    # In happy path we do not expect HITL
    assert answer.route.strategy != "hitl_clarify", "Unexpected HITL clarify in happy path."

    print("Test 1 passed.")
    print("Route:")
    pprint.pprint(answer.route)
    print("\nAnswer:")
    print(answer.answer)    

await test_static_pipeline_clarify_path()



Test 1 passed.
Route:
RouteInfo(used_rag=False,
          used_websearch=False,
          used_tools=False,
          used_user_profile=False,
          used_user_longterm_memory=False,
          strategy='llm_only',
          extra={'attachments_chunks': 0, 'used_attachments_context': False})

Answer:
I couldn't find any information on "Intergrax." It's possible that it may be a fictional or non-existent term, or perhaps it's a company or product that I'm not aware of. If you could provide more context or clarify what you're looking for, I'd be happy to try and help further.


## Test 2 — STATIC pipeline CLARIFY (NEEDS_USER_INPUT → HITL)

This test validates that:

- A deterministic `forced_plan` requiring clarification produces `NEEDS_USER_INPUT`.
- `PlannerStaticPipeline` returns a `RuntimeAnswer` with `route.strategy == "hitl_clarify"`.
- No websearch/tools/rag are used.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime_state import RuntimeState

from intergrax.runtime.nexus.pipelines.planner_static_pipeline import PlannerStaticPipeline

from intergrax.runtime.nexus.planning.engine_plan_models import PlannerPromptConfig, EnginePlan

from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


async def test_static_pipeline_clarify_path():
    # ---------------------------------------------------------------------
    # Config (production components only)
    # ---------------------------------------------------------------------
    cfg = RuntimeConfig(
        llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
        enable_rag=False,
        enable_websearch=False,
        tools_mode="off",
    )

    # If your RuntimeConfig requires explicit StepPlanner/Executor configs (common in your code),
    # you should set them here exactly as in your existing PlanLoopController notebook.
    # This cell is strict: we do not create new fake configs.

    # ---------------------------------------------------------------------
    # Deterministic forced plan (dict, not EnginePlan)
    # Must match EnginePlanner parser contract.
    # ---------------------------------------------------------------------
    cfg.planner_prompt_config = PlannerPromptConfig(
        forced_plan=EnginePlan.clarify(
            version="1",
            reasoning_summary="Deterministic clarify plan for STATIC pipeline integration test.",
            question="Which part should I focus on: architecture, APIs, or runtime execution flow?",
        )
    )


    cfg.plan_loop_policy = PlanLoopPolicy()
    cfg.step_planner_cfg = StepPlannerConfig()
    cfg.step_executor_cfg = StepExecutorConfig()

    # ---------------------------------------------------------------------
    # Runtime context + request + state
    # ---------------------------------------------------------------------
    session_manager = SessionManager(storage=InMemorySessionStorage())

    ctx = RuntimeContext.build(
        config=cfg,
        session_manager=session_manager,
    )

    request = RuntimeRequest(
        user_id="test-user",
        session_id="static-pipeline-e2e-001",
        message="Explain what Intergrax is in one short paragraph.",
        attachments=[],
    )

    state = RuntimeState(
        context=ctx,
        request=request,
        run_id="static-pipeline-e2e-run-001",
    )
    state.configure_llm_tracker()

    # Capabilities ON (avoid planner clamp noise; forced_plan already bypasses LLM planning)
    state.cap_websearch_available = True
    state.cap_tools_available = True
    state.cap_rag_available = True
    state.cap_user_ltm_available = True

    # ---------------------------------------------------------------------
    # Run pipeline
    # ---------------------------------------------------------------------
    pipeline = PlannerStaticPipeline()
    answer = await pipeline.run(state)

    # ---------------------------------------------------------------------
    # Assertions (production meaningful)
    # ---------------------------------------------------------------------
    assert answer is not None, "Pipeline returned None."
    assert state.runtime_answer is answer, "state.runtime_answer was not set to the returned answer."
    assert answer.answer and isinstance(answer.answer, str), "Final answer is empty or invalid."
    assert answer.route is not None, "Route is missing."
    assert answer.route.used_tools is False
    assert answer.route.used_websearch is False
    assert answer.route.used_rag is False
    
    assert answer.route.strategy == "hitl_clarify"
    assert "architecture" in answer.answer.lower() or "which part" in answer.answer.lower()

    print("Test 2 passed.")
    print("Route:")
    pprint.pprint(answer.route)
    print("\nAnswer:")
    print(answer.answer)    

await test_static_pipeline_clarify_path()


Test 2 passed.
Route:
RouteInfo(used_rag=False,
          used_websearch=False,
          used_tools=False,
          used_user_profile=False,
          used_user_longterm_memory=False,
          strategy='hitl_clarify',
          extra={'context_key': 'clarify.user_input',
                 'must_answer_to_continue': True,
                 'origin_step_id': <StepId.CLARIFY: 'clarify'>,
                 'stop_reason': 'needs_user_input'})

Answer:
Which part should I focus on: architecture, APIs, or runtime execution flow?
